In [1]:
import librosa
import pandas as pd
import numpy as np
import os
import pathlib
import csv
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')
import utils
import fma

In [2]:
def build_dataset(tracks, electronic, experimental, folk, hiphop, instrumental, international, pop, rock):
    # Read in genre counts
    genre_counts = {"Electronic": electronic, 
                "Experimental": experimental,
                "Folk": folk,
                "Hip-Hop": hiphop,
                "Instrumental": instrumental,
                "International": international,
                "Pop": pop,
                "Rock": rock
               }
    
    # Build list of tracks to delete
    cnt = np.arange(7997)
    indeces = tracks.index
    delete_indeces = []
    for i in cnt:
        track = tracks.iloc[i, :]
        genre = track["genre_top"]
        if genre_counts[genre] > 0:
            genre_counts[genre] = genre_counts[genre] - 1
        else:
            delete_indeces.append(i)
            
    # Build trimmed copy
    indeces = indeces.delete(delete_indeces)
    new_tracks = tracks.loc[indeces, :]
    genre_check(new_tracks)
    return new_tracks

In [3]:
def extract_features(tracks, new_tracks, features, chroma, rmse, spectral_centroid, spectral_bandwith, spectral_rolloff, zero_crossing_rate, mfcc):
    mask = np.array([True, True, chroma, rmse, spectral_centroid, spectral_bandwith, spectral_rolloff, zero_crossing_rate], dtype=bool)

    if mfcc:
        mfcc_mask = np.ones(20, dtype=bool)
    else:
        mfcc_mask = np.zeros(20, dtype=bool)
        
    mask = np.concatenate((mask, mfcc_mask))
    new_features = features.set_index(tracks.index)
    new_features = new_features.loc[new_tracks.index, mask]
    display(new_features)
    return new_features

In [4]:
def genre_check(data):
    genres = ["Electronic", "Experimental", "Folk", "Hip-Hop", "Instrumental", "International", "Pop", "Rock"]
    count = len(data)
    print(f"Total: {count}")
    
    for i in genres:
        count = len(data.loc[data["genre_top"] == i, :])
        print(f"{i}: {count}")

In [5]:
def trim_data(tracks):
    mask = np.ones(8000, dtype=bool)
    d_mask = pd.Series(mask, index=tracks.index)
    d_mask[99134] = False
    d_mask[108925] = False
    d_mask[133297] = False
    return tracks[d_mask]

In [6]:
def prep_features(features):
    labels = features["label"]
    features = features.drop("label", axis=1)
    return features.insert(1, "labels", labels)

In [7]:
# Load data
full = fma.load('fma/tracks.csv')
small = full[full['set', 'subset'] <= 'small']
artists = small['artist']
tracks = small['track']
trim_tracks = trim_data(tracks)

features = pd.read_csv('fma/fma_small_features.csv')
features = prep_features(features)

In [8]:
w = interactive(build_dataset, {'manual': True}, tracks=fixed(trim_tracks),
                electronic=widgets.IntSlider(min=0, max=999),
                experimental=widgets.IntSlider(min=0, max=999),
                folk=widgets.IntSlider(min=0, max=1000),
                hiphop=widgets.IntSlider(min=0, max=1000),
                instrumental=widgets.IntSlider(min=0, max=1000),
                international=widgets.IntSlider(min=0, max=1000),
                pop=widgets.IntSlider(min=0, max=1000),
                rock=widgets.IntSlider(min=0, max=999));
w.children[8].description="Build dataset"
display(w)

interactive(children=(IntSlider(value=0, description='electronic', max=999), IntSlider(value=0, description='e…

In [11]:
z = interactive(extract_features, {'manual': True},
                tracks=fixed(trim_tracks), new_tracks=fixed(w.result),
                features=fixed(features),
                chroma=False,
                rmse=False,
                spectral_centroid=False,
                spectral_bandwith=False,
                spectral_rolloff=False,
                zero_crossing_rate=False,
                mfcc=False);
z.children[7].description="Extract features"
display(z)

interactive(children=(Checkbox(value=False, description='chroma'), Checkbox(value=False, description='rmse'), …